In [1]:
import numpy as np

In [2]:
%reload_ext autoreload
%autoreload 2

from MTPHandler import Plate, create_standard_curve

from CaliPytion.tools.standardcurve import StandardCurve
from CaliPytion.core import Standard


In [3]:
path = "/Users/max/Documents/enzymeml/alaric/EnzymeML_SLAC_5xConcentration_pH2.0-3.5/ABTS_EnzymeML_340nm_420nm_5x_35deg_pH3.5.txt"
time = np.linspace(0, 15, 31)
time_unit = "min"

# Load plate data
plate = Plate.from_file(path, time, time_unit)

# Define species
buffer = plate.add_reactant(id="s1", name="buffer", constant=True)
abts = plate.add_reactant(id="s0", name="ABTS", constant=False,
                          inchi="InChI=1S/C18H18N4O6S4/c1-3-21-13-7-5-11(31(23,24)25)9-15(13)29-17(21)19-20-18-22(4-2)14-8-6-12(32(26,27)28)10-16(14)30-18/h5-10H,3-4H2,1-2H3,(H,23,24,25)(H,26,27,28)/b19-17-,20-18+")
slac = plate.add_protein(id="p0", name="SLAC",
                         constant=True, sequence="MSSKSKPKDVKV")

# Define well conditions for each species
# Define buffer concentration
plate.assign_species_conditions_to_rows(
    row_ids=["A", "B", "C", "D", "E", "F"],
    species=buffer,
    init_concs=100,
    conc_unit="umol / l"
)

# Define substrate concentrations
plate.assign_species_conditions_to_rows(
    row_ids=["A", "B", "C", "D", "E", "F"],
    species=abts,
    init_concs=[0, 5, 10, 15, 25, 50, 75, 100, 150, 200],
    conc_unit="umol / l"
)

# Define enzyme concentrations
plate.assign_species_conditions_to_rows(
    row_ids=["A", "B", "C"],
    species=slac,
    init_concs=4,
    conc_unit="umol / l"
)

# Blank data
plate.blank_species(buffer, 340)
plate.blank_species(slac, 340)

### Create instance of ```Standard```

In [4]:
print(plate.wells[31])

Well
├── id = D2
├── absorption = [0.08928977792131557, 0.08876732571538556, 0.08803694690943256, 0.08834525527192658, 0.08823467220259057, ...]
├── time = [0.0, 0.5, 1.0, 1.5, 2.0, ...]
├── time_unit = min
├── init_conditions
│   ├── 0
│   │   └── InitCondition
│   │       ├── id = initcondition62
│   │       ├── species_id = s1
│   │       ├── init_conc = 100.0
│   │       ├── conc_unit = umol / l
│   │       └── was_blanked = True
│   └── 1
│       └── InitCondition
│           ├── id = initcondition174
│           ├── species_id = s0
│           ├── init_conc = 5.0
│           ├── conc_unit = umol / l
│           └── was_blanked = False
├── x_position = 1
├── y_position = 3
└── wavelength = 340



In [5]:
print(plate.wells[5])

Well
├── id = A6
├── absorption = [0.8218072149792292, 0.7784879049748431, 0.7466985436004482, 0.7235992754187022, 0.6990941666635572, ...]
├── time = [0.0, 0.5, 1.0, 1.5, 2.0, ...]
├── time_unit = min
├── init_conditions
│   ├── 0
│   │   └── InitCondition
│   │       ├── id = initcondition10
│   │       ├── species_id = s1
│   │       ├── init_conc = 100.0
│   │       ├── conc_unit = umol / l
│   │       └── was_blanked = True
│   ├── 1
│   │   └── InitCondition
│   │       ├── id = initcondition128
│   │       ├── species_id = s0
│   │       ├── init_conc = 50.0
│   │       ├── conc_unit = umol / l
│   │       └── was_blanked = False
│   └── 2
│       └── InitCondition
│           ├── id = initcondition238
│           ├── species_id = p0
│           ├── init_conc = 4.0
│           ├── conc_unit = umol / l
│           └── was_blanked = True
├── x_position = 5
├── y_position = 0
└── wavelength = 340



In [6]:
create_standard_curve(plate, abts, 340, 3).visualize()